In [2]:
import numpy as np
import colorsys
from qiskit import QuantumCircuit
from qiskit.quantum_info import Pauli, SparsePauliOp, Statevector, entropy, partial_trace
import importlib.util
from scipy.stats import circmean

# Chemical exclusive imports
from qiskit import qpy
import json

spec = importlib.util.spec_from_file_location("utils", "effect/utils.py")
utils = importlib.util.module_from_spec(spec)
spec.loader.exec_module(utils)



def get_chemical_colors(initial_angles : list, params : dict):
    """
    Run Chemical model simulation on quantum hardware simulator.

    Args:
        initial_angles (list): List of initial angles (phi, theta) for each drop
        params (dict): Dictionary of parameters for the simulation. Contains two keys, distance and molecule.
                As of now, only H2 is a available as a molecule.

    Returns:
        Final angles after application of circuits.
    """
    n_qubit = len(initial_angles)*2
    print("Initial angles", initial_angles)

    base_circuit = get_circuits_with_vqe(params)[-1] # last_one
    circuit = QuantumCircuit(n_qubit*base_circuit.num_qubits)
    for i in range(n_qubit):
        offset = 2 * i
        qubits = [offset, offset + base_circuit.num_qubits]
        circuit.compose(base_circuit, qubits=qubits, inplace=True)
    
    # Note : not sure about the def of operators
    operators = [SparsePauliOp(Pauli('I'*(n_qubit-i) + p + 'I'*i)) for p in ['X','Y','Z']  for i in range(n_qubit) ]
    obs=utils.run_estimator(circuit, operators, backend=None)

    print("OBSERVABLE", type(obs), obs)
    # first qubit
    x_expectations = [ obs[k//2] for k in range(0,n_qubit,2) ]
    y_expectations = [ obs[k//2] for k in range(n_qubit,2*n_qubit,2) ]
    z_expectations = [ obs[k//2] for k in range(2*n_qubit,3*n_qubit,2) ]

    # phi = arctan2(Y, X)
    phi_expectations = [np.arctan2(y,x) % (2 * np.pi) for x, y in zip(x_expectations, y_expectations)]
    # theta = arccos(Z)
    theta_expectations = [np.arctan2(np.sqrt(x**2 + y**2),z) for x, y, z in zip(x_expectations, y_expectations, z_expectations)]

    final_angles = list(zip(phi_expectations, theta_expectations))

    print("Final angle", type(final_angles), final_angles)
    return final_angles



# The main function using Chemical model
def run(params):
    """
    Executes the effect pipeline based on the provided parameters.

    Args:
        parameters (dict): A dictionary containing all the relevant data.

    Returns:
        Image: the new numpy array of RGBA values or None if the effect failed
    """

    
    # Extract stroke parameters
    image = params["stroke_input"]["image_rgba"]
    assert image.shape[-1] == 4, "Image must be RGBA format"

    height = image.shape[0]
    width = image.shape[1]

    # Extract user-defined parameters
    molecule = params["user_input"]["Molecule"]
    assert molecule == "H2", "Currently only H2 molecule is supported"
    with open('effect/chemical/data/' + molecule.lower() + '_parameters.json', "r") as f:
        circuit_params = json.load(f)
    
    distance = params["user_input"]["Bond Distance"]
    assert 2.5 >= distance >= 0.735, "Distance must be greater than between 0.735 and 2.5 Angstroms"
    distances = [float (d) for d in circuit_params.keys()]
    distance = min(distances, key=lambda x:abs(x-distance))
    
    radius = params["user_input"]["Radius"]
    assert radius > 0, "Radius must be greater than 0"
   
    # Extract other parameters
    path = params["stroke_input"]["path"]
    split_paths = []
    circuits_available = circuit_params[str(distance)]


    initial_angles = [] #(Theta,phi)
    pixels = []
    for lines in split_paths:

        region = utils.points_within_radius(lines, radius, border = (height, width))

        selection = image[region[:, 0], region[:, 1]]
        selection = selection.astype(np.float32) / 255.0
        selection_hls = utils.rgb_to_hls(selection)
    
        phi = circmean(2 * np.pi * selection_hls[..., 0])
        theta = np.pi * np.mean(selection_hls[..., 1], axis=0)
    
        initial_angles.append((phi,theta))
        pixels.append((region, selection_hls))


    final_angles =  damping(initial_angles, strength,invert)

    for i,(region,selection_hls) in enumerate(pixels):
        new_phi, new_theta = final_angles[i]
        old_phi, old_theta = initial_angles[i]

        offset_h = (new_phi - old_phi) / (2 * np.pi)
        offset_l = (new_theta - old_theta) / np.pi

        selection_hls[...,0] = (selection_hls[...,0] + offset_h) % 1
        selection_hls[...,1] += offset_l
    
        #Need to change the luminosity
        selection_hls = np.clip(selection_hls, 0, 1)
        selection_rgb = utils.hls_to_rgb(selection_hls)
        selection_rgb = (selection_rgb * 254).astype(np.uint8)

        image[region[:, 0], region[:, 1]] = selection_rgb
        
        
    return image


In [7]:
import scipy
scipy.__version__

'1.15.3'

In [15]:
with open("effect/chemical/data/h2_circuit.qpy", "rb") as f:
    h2_circuit = qpy.load(f)[0]

with open("effect/chemical/data/h2_parameters.json", "r") as f:
    params = json.load(f)

In [17]:
h2_circuit.assign_parameters({'_t_0_' : params['0.735'][0][0], '_t_1_' : params['0.735'][0][1], '_t_2_' : params['0.735'][0][2]}).draw()

┌───┐┌──────────────────────────┐┌───────────────────────────┐»
q_0: ┤ X ├┤0                         ├┤0                          ├»
     └───┘│                          ││                           │»
q_1: ─────┤1                         ├┤1                          ├»
     ┌───┐│  PauliEvolution(0.24255) ││  PauliEvolution_0(3.0994) │»
q_2: ┤ X ├┤2                         ├┤2                          ├»
     └───┘│                          ││                           │»
q_3: ─────┤3                         ├┤3                          ├»
          └──────────────────────────┘└───────────────────────────┘»
«     ┌─────────────────────────────┐
«q_0: ┤0                            ├
«     │                             │
«q_1: ┤1                            ├
«     │  PauliEvolution_1(-0.62631) │
«q_2: ┤2                            ├
«     │                             │
«q_3: ┤3                            ├
«     └─────────────────────────────┘

In [19]:
params['0.735'][0][1]

3.0994272407908454

In [22]:
list(params.keys())[-1]

'2.5'

In [23]:
mol = 'H2'
with open('effect/chemical/data/' + mol.lower() + '_parameters.json', "r") as f:
    params = json.load(f)

In [26]:
distances = [float (d) for d in params.keys()]
x0 = 1.2
min(distances, key=lambda x: abs(x - x0))

1.1996596596596598